In [ ]:
import numpy as np
import pandas as pd
import gc
import tqdm
import os
import random
import ast
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

epochs = 100
batch_size = 16
margin = 1

In [ ]:
hm_transaction = pd.read_csv('path/transactions_train.csv')
hm_transaction['article_id'] = hm_transaction.article_id.astype(str)
hm_trsc = hm_transaction.drop(['price', 'sales_channel_id'], axis=1)
hm_trsc['t_dat'] = pd.to_datetime(hm_trsc['t_dat'])

hm_article = pd.read_csv('path/articles.csv')
hm_arti = hm_article[['article_id', 'prod_name']]#, 'product_type_name']]
hm_arti['article_id'] = hm_arti.article_id.astype(str)

In [ ]:
hm_test = hm_trsc[hm_trsc.t_dat >='2020-09-01']
hm_val = hm_trsc[(hm_trsc.t_dat < '2020-09-01') & (hm_trsc.t_dat >= '2020-06-01')]
hm_train = hm_trsc[hm_trsc.t_dat < '2020-06-01']

In [ ]:
sim = pd.read_csv('path/img_feat_vgg_cosine.csv')
sim = sim.drop(['Unnamed: 0','top_dist'],axis=1)
sim = sim.drop_duplicates('article_id')

In [ ]:
gc.enable()
del hm_transaction, hm_article
gc.collect()

# Prediction without training

In [ ]:
# Candidate Generation from hm_val set
hm_full = hm_test[['customer_id']]
hm_full = pd.merge(hm_full, hm_val, how='customer_id', how='left')
hm_full = hm_full[['t_dat','customer_id','article_id']]
hm_full = pd.merge(hm_full, sim, on='article_id',how='left')
hm_full = pd.merge(hm_full, hm_arti,on='article_id',how='left' )
hm_full = hm_full[['customer_id','top_sim','prod_name']].rename(columns=
                                                {'top_sim':'article_id'}).dropna(subset='article_id')
# lastest 55 transactions for each customer
trsc_train = hm_full.copy()
trsc_train = trsc_train.groupby('customer_id').tail(55)

count_prod_name = hm_full.groupby(['prod_name','article_id'])['customer_id'].agg('count').reset_index().sort_values(['customer_id'], ascending=False)
top5_prod_ppl = count_prod_name.groupby('prod_name')['article_id'].apply(' '.join).reset_index()
for i in range(len(top5_prod_ppl)):
    top5_prod_ppl.article_id[i] = list(top5_prod_ppl.article_id[i].split())[:30]
prod_train = hm_full.copy()
prod_train = prod_train.drop_duplicates(['customer_id'], keep='last').drop('article_id', axis=1)
prod_train = pd.merge(prod_train, top5_prod_ppl, on='prod_name', how='left')
prod_train = prod_train.explode('article_id')

top5_ppl = hm_full.article_id.value_counts().to_frame().index.astype('str')[:85] #for cold-start customers
top5_ppl_train = hm_full.copy()
top5_ppl_train = top5_ppl_train.drop_duplicates(['customer_id'], keep='last').drop('article_id', axis=1)
top5_ppl_train['article_id'] = [list(top5_ppl) for _ in range(len(top5_ppl_train))]
top5_ppl_train = top5_ppl_train.explode('article_id')

hm_pred = pd.concat([trsc_train, prod_train, top5_ppl_train])
hm_pred = hm_pred.groupby('customer_id').head(85)

In [ ]:
preds = (
    hm_pred
    .groupby('customer_id')[['article_id']]
    .aggregate(lambda x: x.tolist())
)

In [ ]:
test = hm_test.drop(['index','t_dat'],axis=1)
test = test.groupby('customer_id')[['article_id']].aggregate(lambda x: x.tolist())

In [ ]:
prediction = list(preds.article_id)
true = list(test.article_id)

In [ ]:
def recall_at_k(true_labels, pred_labels, k):
    true_positives = 0
    total_positives = len(true_labels)
    if total_positives > k:
        true_labels = true_labels[:k]
    for pred in pred_labels[:k]:
        if pred in true_labels:
            true_positives += 1
    recall = true_positives / total_positives if total_positives > 0 else 0
    return recall

def calculate_recall_at_k(true_labels_list, pred_labels_list, k):
    recall_scores = []
    for true_labels, pred_labels in zip(true_labels_list, pred_labels_list):
        recall = recall_at_k(true_labels, pred_labels, k)
        recall_scores.append(recall)
    mean_recall = sum(recall_scores) / len(recall_scores)
    return mean_recall

def dcg_at_k(r, k):
    r = np.asarray(r)[:k]
    discounts = np.log2(np.arange(len(r)) + 2)
    return np.sum(r / discounts)

def ndcg_at_k(true_labels, pred_scores, k):
    dcg = dcg_at_k(true_labels, k)
    idcg = dcg_at_k(sorted(true_labels, reverse=True), k)
    return dcg / idcg if idcg != 0 else 0

def calculate_ndcg_at_k(y_true, y_pred, k):
    ndcg_scores = []
    for true_items, pred_items in zip(y_true, y_pred):
        relevance = [1 if item in true_items else 0 for item in pred_items]
        ndcg_scores.append(ndcg_at_k(relevance, relevance,k))
    return np.mean(ndcg_scores)

def average_precision_at_k(true_labels, pred_labels, k):
    num_correct = 0
    precision_sum = 0
    for i, pred in enumerate(pred_labels[:k]):
        if pred in true_labels:
            num_correct += 1
            precision_sum += num_correct / (i + 1)
    return precision_sum / min(k, len(true_labels))

def map_at_k(true_labels_list, pred_labels_list, k):
    average_precisions = []
    for true_labels, pred_labels in zip(true_labels_list, pred_labels_list):
        average_precision = average_precision_at_k(true_labels, pred_labels, k)
        average_precisions.append(average_precision)
    return sum(average_precisions) / len(average_precisions)

In [ ]:
print(round(calculate_recall_at_k(true, prediction, 5),5))
print(round(calculate_recall_at_k(true, prediction, 10),5))
print(round(calculate_recall_at_k(true, prediction, 20),5))

print(round(map_at_k(true, prediction, 5),5))
print(round(map_at_k(true, prediction, 10),5))
print(round(map_at_k(true, prediction, 20),5))

print(round(calculate_ndcg_at_k(true, prediction,5),5))
print(round(calculate_ndcg_at_k(true, prediction,10),5))
print(round(calculate_ndcg_at_k(true, prediction,20),5))

# Prediction with Siamese Network

In [ ]:
purchase_dict_4w = {}
hm_4w = hm_val
for i,x in enumerate(zip(hm_4w['customer_id'], hm_4w['article_id'])):
    cust_id, art_id = x
    if cust_id not in purchase_dict_4w:
        purchase_dict_4w[cust_id] = {}
    
    if art_id not in purchase_dict_4w[cust_id]:
        purchase_dict_4w[cust_id][art_id] = 0
    
    purchase_dict_4w[cust_id][art_id] += 1

dummy_list_4w = list((hm_4w['article_id'].value_counts()).index)[:10]

In [ ]:
def prepare_candidates(customers_id, n_candidates = 10):
  """
  df - basically, dataframe with customers(customers should be unique)
  """
  prediction_dict = {}
  dummy_list = list((hm_4w['article_id'].value_counts()).index)[:n_candidates]

  for i, cust_id in enumerate(customers_id):
    # comment this for validation
    if cust_id in purchase_dict_4w:
        l = sorted((purchase_dict_4w[cust_id]).items(), key=lambda x: x[1], reverse=True)
        l = [y[0] for y in l]
        if len(l)>n_candidates:
            s = l[:n_candidates]
        else:
            s = l+dummy_list_4w[:(n_candidates-len(l))]
    else:
        s = dummy_list
    prediction_dict[cust_id] = s

  k = list(map(lambda x: x[0], prediction_dict.items()))
  v = list(map(lambda x: x[1], prediction_dict.items()))
  negatives_df = pd.DataFrame({'customer_id': k, 'negatives': v})
  negatives_df = (
      negatives_df
      .explode('negatives')
      .rename(columns = {'negatives': 'article_id'})
  )
  return negatives_df

In [ ]:
embeddings = pd.read_pickle('hm_embeddings_resnet.pkl')
img_id = pd.read_pickle('filenames.pkl')
img_feat = pd.DataFrame(img_id, columns=['article_id'])
img_feat['article_id'] = [img_feat.article_id[i][1:] for i in range(len(img_feat.article_id))]
img_feat['img_embedding'] = embeddings
img_feat = img_feat.drop_duplicates('article_id')

In [ ]:
df_val = hm_val.drop_duplicates(subset=['customer_id','article_id'])
pos = pd.merge(hm_train, sim, on='article_id', how='left')
pos = pd.merge(pos, df_val,left_on=['customer_id','top_sim'], right_on=['customer_id', 'article_id'], how='inner')
pos = pos.drop(['t_dat_x','t_dat_y', 'article_id_x','article_id_y'],axis=1).rename(columns={'top_sim':'article_id'})

In [ ]:
pos = pos.dropna(subset='article_id')
customer_id = pos['customer_id'].unique()
positives = prepare_candidates(customer_id, 50)
positives = positives.reset_index(drop=True).set_index(['customer_id','article_id'])
positives['label'] = 1

In [ ]:
sim_train = pd.merge(hm_train, sim, on='article_id',how='left')
neg = sim_train[['customer_id','top_sim']].reset_index(drop=True).set_index(['customer_id','top_sim'])
negatives = neg[~neg.index.isin(list(set(positives.index)))].reset_index()
positives = positives.reset_index()
negatives['label'] = 0
negatives = negatives.rename(columns={'top_sim':'article_id'})
negatives = negatives[~negatives.article_id.isna()]
negatives = negatives[negatives.customer_id.isin(list(set(positives.customer_id)))]
negatives = negatives.groupby('customer_id').tail(50)

In [ ]:
sia_full = pd.concat([positives, negatives],ignore_index=True)
sia_full = sia_full.drop_duplicates(['customer_id','article_id'])
sia_full = pd.merge(sia_full, img_feat, on='article_id',how='left')
sia_full = sia_full[~sia_full.img_embedding.isna()]
sia_full['label'] = sia_full.label.astype(int)
sia_full = sia_full.sample(frac=1)
sia_train, sia_val = np.split(sia_full, [int(0.8*len(sia_full))])

In [ ]:
def make_pairs(data):
    pairs = []
    labels = []
    cust_ids = data.customer_id.unique()
    
    for j in tqdm.tqdm(cust_ids):
        sets = data[data.customer_id == j].reset_index(drop=True)
            
        x = sets.img_embedding.reset_index(drop=True)
        y = sets.label.reset_index(drop=True)
        
        num_classes = max(y) + 1
        digit_indices = [np.where(y == i)[0] for i in range(num_classes)]

        for idx1 in range(len(x)):
            # measure cross-group similarity, so if the items belong to the same category, it's consideres as dissimilar
            # add a matching example
            x1 = x[idx1]
            label1 = y[idx1]

            if len(digit_indices[label1]) == 1:
                continue
            idx2 = random.choice(digit_indices[label1])

            x2 = x[idx2]

            pairs += [[x1, x2]]
            labels += [0]
            
            if len(digit_indices) == 1:
                continue
            # add a non-matching example
            label2 = 0 if label1 == 1 else 1
            if len(digit_indices[label2]) == 0:
                continue
            idx2 = random.choice(digit_indices[label2])
            x2 = x[idx2]

            pairs += [[x1, x2]]
            labels += [1]
            #print(j)

    return np.array(pairs), np.array(labels).astype("float32")

In [ ]:
pairs_train, labels_train = make_pairs(sia_train)
pairs_val, labels_val = make_pairs(sia_val)

train_1 = pairs_train[:, 0] 
train_2 = pairs_train[:, 1]
val_1 = pairs_val[:, 0]
val_2 = pairs_val[:, 1]

In [ ]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True)
    return tf.math.sqrt(tf.math.maximum(sum_square, tf.keras.backend.epsilon()))

input_1 = layers.Input(2048)
input_2 = layers.Input(2048)
merge_layer = layers.Lambda(euclidean_distance)([input_1, input_2])
normal_layer = tf.keras.layers.BatchNormalization()(merge_layer)
output_layer = layers.Dense(1, activation="sigmoid")(normal_layer)
siamese = keras.Model(inputs=[input_1, input_2], outputs=output_layer)

def loss(margin=1):
    def contrastive_loss(y_true, y_pred):
        """Calculates the constrastive loss.

        Arguments:
            y_true: List of labels, each label is of type float32.
            y_pred: List of predictions of same length as of y_true,
                    each label is of type float32.

        Returns:
            A tensor containing constrastive loss as floating point value.
        """

        square_pred = tf.math.square(y_pred)
        margin_square = tf.math.square(tf.math.maximum(margin - (y_pred), 0))
        return tf.math.reduce_mean(
            (1 - y_true) * square_pred + (y_true) * margin_square
        )

    return contrastive_loss

siamese.compile(loss=loss(margin=margin), optimizer="RMSprop", metrics=["accuracy"])
siamese.summary()

In [ ]:
history = siamese.fit(
    [train_1, train_2],
    labels_train,
    validation_data=([val_1, val_2], labels_val),
    batch_size=batch_size,
    epochs=epochs,
)

In [ ]:
def plt_metric(history, metric, title, has_valid=True):
    plt.plot(history[metric])
    if has_valid:
        plt.plot(history["val_" + metric])
        plt.legend(["train", "validation"], loc="upper left")
    plt.title(title)
    plt.ylabel(metric)
    plt.xlabel("epoch")
    plt.show()


# Plot the accuracy
plt_metric(history=history.history, metric="accuracy", title="Model accuracy")

# Plot the constrastive loss
plt_metric(history=history.history, metric="loss", title="Constrastive Loss")

In [ ]:
group_a = hm_pred[['customer_id']].drop_duplicates(subset='customer_id')
purchased = hm_val.groupby(['customer_id','article_id']).t_dat.count().reset_index().sort_values(
                                                                't_dat',ascending=False)
purchased = purchased.groupby(['customer_id']).head(5)
purchased = purchased[purchased.customer_id.isin(list(group_a.customer_id))]
group_a = pd.merge(group_a,purchased,on='customer_id',how='left')

# find most purchased item for cold_start customers
ppl = hm_val.article_id.value_counts().to_frame().index.astype('str')[1:5]
ppl_df = hm_pred[['customer_id']].drop_duplicates(subset='customer_id')
ppl_df['article_id'] =[ list(ppl) for _ in range(len(ppl_df))]
ppl_df = ppl_df.explode('article_id')

group_a = pd.concat([group_a,ppl_df],ignore_index=True)
group_a = group_a[group_a.article_id.isin(list(img_feat.article_id))]

group_a = group_a.merge(img_feat, on='article_id',how='left')
hm_pred = hm_pred.merge(img_feat,on='article_id',how='left')

In [ ]:
def make_test_pairs(data,group_b):
    pairs = []
    labels = []
    cust_ids = data.customer_id.unique()
    
    for j in tqdm.tqdm(cust_ids):
        sets_a = data[data.customer_id == j].reset_index(drop=True)
        sets_b = group_b[group_b.customer_id == j].reset_index(drop=True)
            
        x_a = sets_a.img_embedding
        x_b = sets_b.img_embedding
        #y = sets.label

        #digit_indices = np.where(y == 1)[0] 

        for idx1 in range(len(x_a)):
            for idx2 in range(len(x_b)):
                # measure cross-group similarity, so if the items belong to the same category, it's consideres as dissimilar
                # add a matching example
                x1 = x_a[idx1]
                x2 = x_b[idx2]

                pairs += [[x1, x2]]
                labels += [0]

    return np.array(pairs), np.array(labels).astype("float32")

In [ ]:
pairs_test, labels_test = make_test_pairs(group_a, hm_pred)
test_1 = pairs_test[:, 0]
test_2 = pairs_test[:, 1]

In [ ]:
predictions = []
batch_size = 5000
for bucket in tqdm.tqdm(range(0, len(test_1), batch_size)):
  outputs = siamese.predict(
      [test_1[bucket: bucket+batch_size], test_2[bucket: bucket+batch_size] ]
      )
  predictions.append(outputs)
predictions = np.concatenate(predictions)
len(prediction)

In [ ]:
sia_pred = hm_pred.copy()
sia_pred['pred'] = predictions
sia_pred = sia_pred.sort_values(['customer_id','pred'],ascending=False)
sia_pred = sia_pred.groupby(['customer_id','article_id']).prod_name.agg('count').reset_index().sort_values('prod_name',ascending=False)
sia_pred = sia_pred.groupby('customer_id').head(20)

In [ ]:
sia_pred_lst = (
    sia_pred
    .groupby('customer_id')[['article_id']]
    .aggregate(lambda x: x.tolist())
)
prediction = list(sia_pred_lst.article_id)

In [ ]:
print(round(calculate_recall_at_k(true, prediction, 5),5))
print(round(calculate_recall_at_k(true, prediction, 10),5))
print(round(calculate_recall_at_k(true, prediction, 20),5))

print(round(map_at_k(true, prediction, 5),5))
print(round(map_at_k(true, prediction, 10),5))
print(round(map_at_k(true, prediction, 20),5))

print(round(calculate_ndcg_at_k(true, prediction,5),5))
print(round(calculate_ndcg_at_k(true, prediction,10),5))
print(round(calculate_ndcg_at_k(true, prediction,20),5))